<a href="https://colab.research.google.com/github/pangeab-blip/EvGeo-Exercises/blob/main/Migration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# ============================================================
# Climate-Window Migration Model (teaching version, annotated)
# ============================================================
# PURPOSE
# A simple two-box model: Origin (O) and Destination (D).
# Populations grow logistically; migration occurs from O → D
# when a climate "corridor" is open.
#
# VARIABLES AND PARAMETERS
# ------------------------------------------------------------
# Population dynamics:
#   O(t), D(t)    : population sizes in Origin and Destination (individuals)
#   O0, D0        : initial populations (individuals)
#   rO, rD        : intrinsic growth rates (per year, 1/yr).
#                   This is the maximum fractional growth rate when population is small.
#                   Example: rO = 0.02 → 2% increase per year at low density.
#   KO, KD        : carrying capacities (individuals). Maximum sustainable populations.
#
# Migration:
#   m_base        : baseline migration rate (fraction of Origin per year if corridor open).
#                   Example: m_base=0.01 → 1% of the Origin population migrates each year
#                   while the corridor is open.
#   M(t)          : migration flow (individuals per timestep Δt).
#
# Climate corridor:
#   C(t)          : climate index (dimensionless), controls whether the corridor is open.
#                   Defined as: C(t) = C0 + A*sin(2πt/P + φ) + shock(t).
#   C0            : baseline climate index (dimensionless).
#   A             : amplitude of oscillation (dimensionless).
#   P             : period of oscillation (years). Sets the climate cycle length.
#   φ (phi_deg)   : phase shift (degrees). Shifts the sine wave along the time axis.
#
# Threshold function:
#   T(t)          : threshold (dimensionless) the climate must exceed for corridor to open.
#                   Defined as: T(t) = T0 + trend * t
#   T0            : baseline threshold value (dimensionless).
#   trend         : drift of threshold per year (1/yr). Represents long-term change (e.g. tectonic uplift).
#
# Corridor condition:
#   Corridor is open if C(t) ≥ T(t).
#
# Shock event:
#   shock(t)      : Gaussian perturbation to C(t).
#   shock_year    : center year of the shock (years).
#   shock_width   : width of the Gaussian (years).
#   shock_amp     : amplitude of the shock (positive = opens corridor more easily, negative = closes).
#
# Time:
#   years         : total duration of simulation (years).
#   dt            : timestep (years).
#
# OUTPUTS
# - Populations O(t), D(t) over time.
# - Climate index C(t) vs threshold T(t), shaded regions show when corridor is open.
# - Migration flow M(t)/dt (individuals per year).
# ============================================================
import numpy as np
import math
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Checkbox

def migration_model(
    years=10000,           # durata simulazione (anni)
    dt=10,                 # passo (anni)
    # Popolazioni iniziali e parametri demografici
    O0=1e4, D0=100.0,      # popolazioni iniziali (individui)
    rO=0.02, rD=0.015,     # tassi di crescita annua
    KO=1e6, KD=5e6,        # capacità portante (carrying capacity)
    # Migrazione
    m_base=0.01,           # frazione/anno che migra se corridoio aperto
    # Clima/proxy del corridoio
    C0=0.0,                # media dell'indice climatico
    A=1.0,                 # ampiezza
    P=2000.0,              # periodo (anni)
    phi_deg=0.0,           # fase (gradi)
    T0=0.0,                # soglia media
    trend=0.0,             # tendenza della soglia per anno
    # Shock singolo (es. abbassamento livello marino: corridoio più aperto)
    shock_year=3000.0,     # anno centro shock
    shock_width=500.0,     # durata caratteristica (deviazione gaussiana)
    shock_amp=0.0,         # ampiezza shock sul clima (+ apre, − chiude)
    # Opzioni grafiche
    show_flux=True,
    ylog=False
):
    # griglia temporale
    years = int(years)
    dt = float(dt)
    t = np.arange(0, years+dt, dt)
    n = t.size

    # preallocazioni
    O = np.zeros(n); D = np.zeros(n); M = np.zeros(n)
    C = np.zeros(n); Tthr = np.zeros(n); open_flag = np.zeros(n, dtype=int)

    # stato iniziale
    O[0] = max(O0, 0.0)
    D[0] = max(D0, 0.0)

    # componenti climatiche
    phi = math.radians(phi_deg)
    # shock gaussiano (positivo apre il corridoio)
    shock = shock_amp * np.exp(-0.5*((t - shock_year)/max(shock_width,1e-9))**2)

    # costanti utili
    two_pi_over_P = 2.0*math.pi/max(P,1e-9)

    # simulazione
    open_years = 0.0
    for i in range(n):
        # indice climatico e soglia
        C[i] = C0 + A*np.sin(two_pi_over_P*t[i] + phi) + shock[i]
        Tthr[i] = T0 + trend*t[i]
        open_flag[i] = 1 if C[i] >= Tthr[i] else 0
        if i>0:
            open_years += open_flag[i]*dt

        if i == n-1:  # ultimo passo: non aggiornare
            break

        # crescita logistica
        dO_growth = rO*O[i]*(1.0 - O[i]/max(KO,1e-12))*dt
        dD_growth = rD*D[i]*(1.0 - D[i]/max(KD,1e-12))*dt

        # migrazione (limitata dall'Origine e dal corridoio)
        M[i] = m_base * O[i] * open_flag[i] * dt

        # aggiornamenti
        O[i+1] = max(O[i] + dO_growth - M[i], 0.0)
        D[i+1] = max(D[i] + dD_growth + M[i], 0.0)

    # statistiche
    total_migrants = np.sum(M)   # numero cumulato di individui migrati
    frac_out = total_migrants / max(O[0] + np.sum(rO*O*(1 - O/np.maximum(KO,1e-12))*dt), 1e-9)  # proxy (grezzo)
    open_fraction = open_years / (t[-1] - t[0])

    # riepilogo
    print("=== Climate-Window Migration Model ===")
    print(f"Time span: 0–{int(t[-1])} yr, Δt={dt} yr  |  Corridor open ~{open_fraction*100:.1f}% of time")
    print(f"Initial populations: Origin={O0:.0f}, Destination={D0:.0f}")
    print(f"Growth: rO={rO:.3f} 1/yr, KO={KO:.0f}  |  rD={rD:.3f} 1/yr, KD={KD:.0f}")
    print(f"Migration: m_base={m_base:.3f} 1/yr when open")
    print(f"Climate index C(t)=C0+A*sin(2πt/P+φ)+shock with C0={C0:.2f}, A={A:.2f}, P={P:.0f} yr, φ={phi_deg:.0f}°")
    print(f"Threshold T(t)=T0+trend*t with T0={T0:.2f}, trend={trend:.2e} /yr; shock_amp={shock_amp:.2f} at year ~{shock_year:.0f}")
    print(f"Total migrants (cumulative): {total_migrants:.0f}")

    # --- Grafico 1: Popolazioni ---
    plt.figure(figsize=(7,4))
    if ylog:
        plt.semilogy(t, np.maximum(O,1e-9), label="Origin")
        plt.semilogy(t, np.maximum(D,1e-9), label="Destination")
        plt.ylabel("Population (log)")
    else:
        plt.plot(t, O, label="Origin")
        plt.plot(t, D, label="Destination")
        plt.ylabel("Population")
    plt.xlabel("Time (yr)")
    plt.title("Population trajectories")
    plt.legend()
    plt.grid(True, linestyle=":")
    plt.show()

    # --- Grafico 2: Clima e soglia, e finestra di migrazione ---
    plt.figure(figsize=(7,4))
    plt.plot(t, C, label="Climate index C(t)")
    plt.plot(t, Tthr, label="Threshold T(t)")
    # evidenziare finestre aperte
    if np.any(open_flag==1):
        # riempi dove open
        mask = open_flag==1
        plt.fill_between(t, C, Tthr, where=mask, alpha=0.2, step="pre")
    plt.xlabel("Time (yr)")
    plt.ylabel("Index / Threshold (arb.)")
    plt.title("Corridor opening: C(t) vs T(t)  (shaded = open)")
    plt.legend()
    plt.grid(True, linestyle=":")
    plt.show()

    # --- Grafico 3 (opzionale): flusso migratorio per passo ---
    if show_flux:
        plt.figure(figsize=(7,3.6))
        plt.plot(t[:-1], M[:-1]/dt, label="Migration flow (ind/yr)")
        plt.xlabel("Time (yr)")
        plt.ylabel("Migrants per year")
        plt.title("Instantaneous migration flow")
        plt.grid(True, linestyle=":")
        plt.show()

    return {
        "time_yr": t,
        "Origin": O,
        "Destination": D,
        "Migrants_per_step": M,
        "ClimateIndex": C,
        "Threshold": Tthr,
        "OpenFlag": open_flag,
        "OpenFraction": open_fraction,
        "TotalMigrants": total_migrants
    }

# UI
interact(
    migration_model,
    years=IntSlider(value=10000, min=1000, max=50000, step=500, description="Years"),
    dt=IntSlider(value=10, min=1, max=100, step=1, description="Δt (yr)"),
    O0=FloatSlider(value=1e4, min=10.0, max=1e7, step=10.0, description="Origin_0"),
    D0=FloatSlider(value=100.0, min=0.0, max=1e7, step=10.0, description="Dest_0"),
    rO=FloatSlider(value=0.02, min=0.0, max=0.1, step=0.001, description="r_O (1/yr)"),
    rD=FloatSlider(value=0.015, min=0.0, max=0.1, step=0.001, description="r_D (1/yr)"),
    KO=FloatSlider(value=1e6, min=1e3, max=1e8, step=100.0, description="K_O"),
    KD=FloatSlider(value=5e6, min=1e3, max=1e8, step=100.0, description="K_D"),
    m_base=FloatSlider(value=0.01, min=0.0, max=0.2, step=0.001, description="m_base (1/yr)"),
    C0=FloatSlider(value=0.0, min=-2.0, max=2.0, step=0.01, description="C0"),
    A=FloatSlider(value=1.0, min=0.0, max=5.0, step=0.01, description="Amplitude A"),
    P=FloatSlider(value=2000.0, min=100.0, max=20000.0, step=10.0, description="Period P (yr)"),
    phi_deg=FloatSlider(value=0.0, min=0.0, max=360.0, step=1.0, description="Phase φ (deg)"),
    T0=FloatSlider(value=0.0, min=-2.0, max=2.0, step=0.01, description="T0 (threshold)"),
    trend=FloatSlider(value=0.0, min=-1e-3, max=1e-3, step=1e-5, description="Trend (/yr)"),
    shock_year=FloatSlider(value=3000.0, min=0.0, max=50000.0, step=10.0, description="Shock year"),
    shock_width=FloatSlider(value=500.0, min=10.0, max=5000.0, step=10.0, description="Shock width"),
    shock_amp=FloatSlider(value=0.0, min=-3.0, max=3.0, step=0.01, description="Shock amp"),
    show_flux=Checkbox(value=True, description="Show flow"),
    ylog=Checkbox(value=False, description="Y log (pop)"),
);

interactive(children=(IntSlider(value=10000, description='Years', max=50000, min=1000, step=500), IntSlider(va…